In [2]:
from align_timestamps import TranscribedAudio
from glob import glob
from pathlib import Path
import pandas as pd


In [16]:
# align all transcriptions with audio and save alignments so as to not do alignment everytime
def align_all(load_saved=True):
    model = TranscribedAudio.load_model('tiny')
    filenames = []
    ats = {} # aligned transcriptions
    for idx,folder in enumerate(Path('../data/unorganized/').iterdir()):
        if str(folder.parts[-1]).startswith('.'):
            continue
        wav = glob(str(folder.joinpath( '*.wav')))
        if len(wav) != 1:
            print('problem locating wav in this folder: ',folder)
            continue
        else:
            wav = wav[0]
        textgrid = glob(str(folder.joinpath( '*Fadhl*.TextGrid')))
        if len(textgrid) != 1:
            print('problem locating textgrid in this folder: ',folder)
            continue
        else:
            textgrid = textgrid[0]
        wav_path = '/'.join(Path(wav).parts[-2:])
        textgrid_path = '/'.join(Path(textgrid).parts[-2:])
                    
        filenames.append((wav_path, textgrid_path))

        ats[idx] = TranscribedAudio(wav, textgrid)
        if load_saved and Path(f'../data/saved_alignments/{Path(textgrid).parts[-1]}.tsv').exists():
            ats[idx].load_aligned_words(f'../data/saved_alignments/{Path(textgrid).parts[-1]}.tsv')
        else:
            
            ats[idx].model = model
            ats[idx].align_timestamps(language='ar')
            ats[idx].save_aligned_words(f'../data/saved_alignments/{Path(textgrid).parts[-1]}.tsv')
            
    
    filenames_df = pd.DataFrame(filenames, columns=['wav', 'textgrid'])
    filenames_df.to_csv('../data/filenames.tsv', sep='\t', index=False)
    return ats

ats = align_all(load_saved=True)

Align: 100%|██████████| 2712.93/2712.93 [00:29<00:00, 92.49sec/s] 
/Users/f/ba3sasah/khaafis/src/align_timestamps.py:81: UserWarning: 62/1021 segments failed to align.
  aligned_result = self.model.align(self.wav_path, full_transcript, language)
Align: 100%|██████████| 3333.76/3333.76 [00:44<00:00, 74.49sec/s] 
/Users/f/ba3sasah/khaafis/src/align_timestamps.py:81: UserWarning: Failed to align the last 2/5408 words after 55:33.740.
  aligned_result = self.model.align(self.wav_path, full_transcript, language)
/Users/f/ba3sasah/khaafis/src/align_timestamps.py:81: UserWarning: 61/968 segments failed to align.
  aligned_result = self.model.align(self.wav_path, full_transcript, language)
Align: 100%|██████████| 1756.83/1756.83 [00:13<00:00, 130.07sec/s]
/Users/f/ba3sasah/khaafis/src/align_timestamps.py:81: UserWarning: 37/545 segments failed to align.
  aligned_result = self.model.align(self.wav_path, full_transcript, language)
Align: 100%|██████████| 3115.65/3115.65 [00:43<00:00, 71.94sec/s

In [17]:
#join everything into a single dataframe
dfs = []
for key in ats.keys():    
    df = ats[key].aligned_words
    df['file_idx'] = key
    dfs.append(df)
wordsdf = pd.concat(dfs).dropna(subset=['word']).reset_index(drop=True)


In [161]:
# helper functions to query words and get audio snippets
def get_word(word,context=3,highlight=True):
    q = wordsdf[wordsdf['word'].str.contains(word,regex=True)].copy()
    for idx in q.index:
        q.loc[idx,'context'] = wordsdf.loc[idx-context:idx+context,'word'].str.cat(sep=' ')   
        if highlight:   
            q.loc[idx,'context'] = q.loc[idx,'context'].replace(word, f'**{word}**') 
        
    return q.reset_index(names=['word_idx'])

def get_audio(q, index=0,audio_pad_margin=(-1, 1)):
    print('word:',q['word'].values[index])
    print('context:',q['context'].values[index])
    return ats[q['file_idx'].values[index]].get_audio(q, index=index, audio_pad_margin=audio_pad_margin) # class specific TranscribedAudio.get_audio()

In [37]:
q = get_word('ما\s?في'+'|'+'ما\s?شي')
q

,word,start,end,tiers,file_idx,context
0,ماشي,1184.12,1184.48,Manager,4,%أأ عامية. العامية. ماشي ما في مشكلة
1,ماشي,1275.50,1275.84,S2|S1,7,+فstay tuned يمكن ماشي mask من ما
2,مافي,1715.34,1715.64,M,7,نفسي أفضل العامية مافي مشكلة بس أنا
3,ومافي,2397.52,2398.28,S1,7,تكون شخصية جدا ومافي teamwork مافي يعني
4,مافي,2399.16,2399.62,S1,7,جدا ومافي teamwork مافي يعني عمل كفرقة
5,ماشي,1508.06,1508.30,Manager,15,الأسئلة التي حضرتوها. ماشي الحال؟ هيه. هيه
6,ماشي,1511.10,1511.10,S1|S2,15,هيه. هيه %هم ماشي %أأ. ماشي الحال
7,ماشي,1511.10,1511.10,S1|S2,15,%هم ماشي %أأ. ماشي الحال (( )).


In [39]:
# /m aa f i y/
get_audio(q, index=3,audio_pad_margin=(-1, 2))

word: ومافي
context: تكون شخصية جدا ومافي teamwork مافي يعني


In [42]:
q = get_word('محد', context=2)
q

,word,start,end,tiers,file_idx,context
0,المحددة,87.680,88.080,S1,1,بعض النصائح ال**محد**دة حول الأماكن
1,المحددة,114.760,115.420,S2,1,بعض الأنواع ال**محد**دة من المعلومات
2,المحدد,2678.420,2678.720,S1,4,في هاليوم ال**محد**د والوقت الفلاني،
3,محدودة,1680.120,1680.660,Manager,6,التي أستهدفها **محد**ودة إلى حد
4,ومحد,482.780,483.260,S1,8,عندها clients و**محد** يعرف عنها
5,محد,619.960,620.400,S1,9,البدويين إن **محد** يقدر يكلمهم
6,محد,623.180,623.480,S1,9,بروحهم إن **محد** يقدر يتدخل
7,المحدود,2580.264,2580.924,S2,11,علامة الإصدار ال**محد**ود أو الإصدار
8,المحدود.,2606.444,2606.944,S2,11,علامة الإصدار ال**محد**ود. طب، ممكن
9,المحدود؟,2610.664,2612.304,Manager,11,المقصود بالإصدار ال**محد**ود؟ إنه إذا


In [45]:
# /m a 7 7 a d/
get_audio(q, index=6, audio_pad_margin=(-1, 1))

word: محد
context: بروحهم إن **محد** يقدر يتدخل


In [68]:
q = get_word('شاسمه'+'|'+'شو',context=2)
q = q[q['context'].str.contains('شو اسمه'+'|'+'شاسمه')].reset_index(drop=True)
q

,word,start,end,tiers,file_idx,context
0,شو,449.08,449.18,S2,6,هذي سلوى شو تريد؟ %أأ


In [67]:
# 5 - /sh u s m u h u m/
# 6 - /sh u s m a h/
# 10 - /sh e s m a h/
get_audio(q, index=6, audio_pad_margin=(-1, 1))

word: شو
context: يقدرون يروحون شو اسمه ..


In [ ]:
q = get_word('وحدة',context=2)
q

,word,start,end,tiers,file_idx,context
0,وحدة,2802.24,2802.34,S1,7,تسعة لين **وحدة** هذا بداية
1,وحدة,2978.98,2979.36,S1,7,بسرعة تكون **وحدة** ورا **وحدة**
2,وحدة,2979.54,2979.80,S1,7,**وحدة** ورا **وحدة** يعني سريعة
3,وحدة,973.08,973.18,S1,9,عادي مثلا **وحدة** تكون وياها..
4,وحدة,892.54,892.82,S2,14,كيف كل **وحدة** ال +weakness
5,وحدة.,1095.64,1095.74,S2,14,ولا دولة **وحدة**. إنه قالوا
6,وحدة,1775.24,1775.58,S1,14,لقى قطعة **وحدة** يقول الفرص
7,وحدة,1824.12,1825.00,S1,14,لأنه فيه **وحدة** %أأ ..
8,وحدة،,2014.18,2014.58,S1,14,فروع مرة **وحدة**، أو مثلا
9,وحدة.,2016.00,2016.10,S1,14,فروع مرة **وحدة**. بتكون يعني


In [41]:
# /w i 7 d a/
get_audio(q, index=5, audio_pad_margin=(-1, 1))

word: وحدة.
context: ولا دولة **وحدة**. إنه قالوا


In [74]:
pd.set_option('display.max_colwidth', 100)

In [173]:

q = get_word('(مناك|مني)', context=4)
q

/var/folders/nb/rn_wr53j4mbcyl_kw5fw8t3m0000gn/T/ipykernel_630/2634808447.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  q = wordsdf[wordsdf['word'].str.contains(word,regex=True)].copy()


,word_idx,word,start,end,tiers,file_idx,context
0,6888,الزمنية,970.88,971.44,S2,1,عن %إم متى الفترة الزمنية اللي بيجون فيها. %إم
1,7235,مناك،,1356.54,1356.64,S2,1,وكوفيات الناس تقدر تجلس مناك، وأكثر شي فأيام شتاء
2,20813,مني,901.38,901.92,S2,5,مطاعم بس مطاعم شي مني مثل برع. {pause} %إهم
3,82113,مني,2853.92,2854.22,S2,15,تعرف أو تفهم أكثر مني بحكم إن هي طالبة


In [176]:
# 1 - /m i n n aa k/
# 2 - /m i n n i y/
get_audio(q, index=1, audio_pad_margin=(-1, 1))

word: مناك،
context: وكوفيات الناس تقدر تجلس مناك، وأكثر شي فأيام شتاء


In [137]:
q = get_word('^ا?خذ', context=4)
q

,word_idx,word,start,end,tiers,file_idx,context
0,15350,خذنا,940.28,940.72,S2,4,topics? Special topics بس خذنا شوي يعني %أأ recycling
1,15381,خذتي,959.66,959.92,S1,4,Okay. هاي شو كان؟ خذتي special to-- ~ لا
2,15472,اخذت,1001.78,1002.10,S1,4,يمكن باخذ photography. أنا اخذت هال +program في هال
3,15639,خذتي,1113.13,1113.25,S1,4,%آه أوكي. إلا إذا خذتي ال +art foundations وهالسوالف.
4,15647,خذت.,1118.41,1118.51,S2,4,وهالسوالف. Art foundation هيه خذت. %إم يمكن عيل تصدقي.
5,34119,خذت,1217.12,1217.38,S2,7,أعرف شي عنكم وما خذت ولا elective فاحتاجج تفهميني
6,34955,اخذتي,1582.36,1582.78,S2,7,إمم % إنتي شو اخذتي last semester? خذت two
7,34958,خذت,1584.70,1585.02,S1,7,شو اخذتي last semester? خذت two one two أتوقع
8,35005,اخذتي,1599.96,1600.94,S1,7,twenty هيه ويا Jigar اخذتي وياي؟ اخذت Mental health
9,35007,اخذت,1601.50,1601.90,S2,7,ويا Jigar اخذتي وياي؟ اخذت Mental health psyc three


In [142]:
# 15350 (0) - /kh a dh n a/
# 34958 (7) and 78436 (36) - /kh a dh t/
# 42371 (19) /2 i l l i # 2 a kh dh ee n aa h a/
#  78002 (28) /2 i l l i # kh a dh ee n aa h/

get_audio(q, index=14, audio_pad_margin=(-2, 2))

word: خذينا
context: +oil. يعني بهالمادة نحن خذينا إن دبي مثلا ما


In [154]:
q = get_word('هذ(ي|ه)$', context=4)
q[q['tiers'].str.contains('m|M')]

/var/folders/nb/rn_wr53j4mbcyl_kw5fw8t3m0000gn/T/ipykernel_630/1721353410.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  q = wordsdf[wordsdf['word'].str.contains(word,regex=True)].copy()


,word_idx,word,start,end,tiers,file_idx,context
2,8268,هذه,2212.460,2212.700,Manager,1,في الشارقة. %إم. طيب، هذه معلومات قيمة وكثيرة في
3,8553,هذه,2349.480,2349.480,Manager,1,في خلال ال-- %أأ هذه الأشهر، فهل فيه فعاليات
5,8802,هذه,2479.500,2479.800,Manager,1,في أبو ظبي. كل هذه الصور على الشاشة كلها
9,9093,هذه,2641.760,2642.240,Manager,1,أشياء أخرى .. وما هذه الصورة هنا؟ GULF -MSA
13,9409,هذه,2829.940,2830.240,Manager,1,ليزورها. %أأ فمن كل هذه الأماكن هل هناك مثلا
15,9533,هذه,2889.320,2889.700,Manager,1,إسبوعين، ((هذ--)) الثلاث مدن هذه أو الثلاث ولايات %أأ
16,9782,هذه,3021.200,3021.460,Manager,1,في أماكن معينة، فهل هذه الأمور أيضا سلسة أو
18,17555,هذه,2216.580,2216.580,Manager,4,إيه أفكارك عن %أأ هذه الحملة. الآن كل من--
23,28942,هذه,1719.720,1719.980,Manager,6,بتصنيعها ذات جودة عالية، هذه هي الميزة الأساسية وربما
25,29401,هذه,1991.540,1991.820,Manager,6,رخصة تجارية. %أأ .. هذه هذه نقاط جيدة جدا


In [159]:
# 8268 (2) /h a dh i h i/
# 76983 (81) /h aa dh i/
get_audio(q, index=81, audio_pad_margin=(-3, 2))

word: هذه
context: يعني يفت-- ~ %أها هذه الشركات دولية كبيرة جدا.


In [164]:
q = get_word('بنقول', context=4,highlight=False)
q[q['context'].str.contains('بنقول ل')]

,word_idx,word,start,end,tiers,file_idx,context
2,7216,بنقول,1347.340,1347.620,S2,1,بنقول عن مرسى البطين، بنقول لهم إنه يعني فيه
3,7312,بنقول,1434.100,1434.540,S2,1,{laugh}. يعني أو إحنا بنقول لها يمكن حتى الأشياء
4,7318,بنقول,1436.860,1437.200,S2,1,يمكن حتى الأشياء اللي بنقول لها ما بتكون ((هي))
6,40422,بنقول,1147.512,1147.672,S2,8,sustainable. Locally prodcued .. بنقول لها إنه تسوي هالشي،


In [172]:
# 7312 and 7318 (3 and 4) /b e n g uu l # i l h a/
# 40422 (6) /b a n g u l h a/
get_audio(q, index=6, audio_pad_margin=(-2, 2))

word: بنقول
context: sustainable. Locally prodcued .. بنقول لها إنه تسوي هالشي،
